# Cálculo de error entre modelos y optimización de parámetros.

La idea es hacer una función que tome los parámetros del modelo y pueda calcular el error entre el modelo generado y uno de referencia dado.

e(x) = [x,t,beta,gamma]

dónde x es un vector con los valores iniciales de los compartimentos.

Se establece una función que permite calcular el error de los casos diarios entre dos modelos, uno de referencia y otro el modelo desarrollado.

Una vez calculado este error, se utiliza la opción minimize en busca de minimizarlo cambiando los parámetros del modelo. En el caso de SIR, betta y gamma.

De esta manera, puede plantearse un modelo con parámetros estimativos y luego encontrar los parámetros que mejor ajustan al modelo a los datos de referencia.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import minimize

# Valores iniciales.

In [ ]:
N = 1    # Total population
I_0 = 1/1e6     # Initial infected population
S_0 = N-I_0  # Initial susceptible population (exposed)
R_0 = 0      # Initial recovered population
x0 = [S_0, I_0, R_0]

# Definición del modelo SIR.

In [ ]:
def SIR_model(x,t, beta, gamma): # Toma el vector x = [S,I,R] y el tiempo.
    S, I, R = x

    dS_dt = - beta * I * S/N
    dI_dt = beta * I * S / N  - gamma * I
    dR_dt = gamma*I
    return [dS_dt, dI_dt, dR_dt]

# Simulación del tiempo.

In [ ]:
t_start = 0.0
t_end = 100.0
num_points = 1000
time_points = np.linspace(t_start, t_end, num_points) # Genera 1000 puntos entre 0 y 100 equiespaciados.

# Parámetros conocidos de referencia.

In [ ]:
# Creamos la solución real, de referencia
betaR = 1
gammaR = 0.5

# Estimación inicial de parámetros

In [ ]:
#Parámetros estimados a ser encontrados por minimize.
betaE = 0.7
gammaE = 0.2

# Función que calcula los casos diarios.

Cabe destacar que solo importará el error de la cantidad de gente que se va infectando. Para calcular esto, notemos que la ecuación del compartimento S es:

$dS/dt = -β * S * I$

Luego, podemos deducir que el - de la ecuación se debe a que la misma representa a la tasa de personas que dejan de ser susceptibles. Es decir, la tasa de personas que se van infectando.

Entonces lo que vamos a querer calcular como error es la diferencia de ambos modelos en la ecuacion:

$β*S*I$

In [ ]:
def dailyCases(beta,gamma):
  sol = odeint(SIR_model, x0, time_points,rtol=1e-6, atol=1e-12,args=(beta,gamma))
  S = sol[:, 0]
  I = sol[:, 1]
  R = sol[:, 2]
  return beta*S*I/N

# Precalculo de los casos diarios de referencia.

In [49]:
dailyRef = dailyCases(betaR, gammaR)

# Función del error.

Para calcular el error se hace la norma L2 (euclideana) entre los casos diarios predecidos y los reales. Luego se normaliza con el error de los casos reales para no entrar en errores de escala.




In [48]:
def errorFun(params):

    # Sacamos los parámetros.
    # Estos luego serán minimizados por la función minimize.
    beta, gamma = params

    dailyPredict = dailyCases(beta, gamma)

    errRealL2 = np.linalg.norm(dailyRef)
    errL2 = np.linalg.norm(dailyPredict - dailyRef)

    return errL2/errRealL2

# Cálculo del error.

In [ ]:
# Calculamos el error entre los datos reales y el modelo con los parámetros
# propuestos.
errL2 = errorFun([betaE,gammaE])
print(errL2)

0.31046752417104945


# Minimización del error y búsqueda de parámetros.

In [ ]:
## Minimizamos la función en busca de beta y gamma correctos, es decir,
# que más se acerquen a los resultados del modelo de referencia.
result = minimize(errorFun, x0=[betaE, gammaE], method='BFGS')

/tmp/ipython-input-3165794217.py:2: ODEintWarning: Repeated error test failures (internal error). Run with full_output = 1 to get quantitative information.
  sol = odeint(SIR_model, x0, time_points,rtol=1e-6, atol=1e-12,args=(beta,gamma))


# Impresión de valores finales.

In [ ]:
# Imprimimos resultados.
print("Minimum found at:", result.x)
print("Minimum function value:", result.fun)
print("Optimization successful:", result.success)

Minimum found at: [0.99999992 0.49999992]
Minimum function value: 6.277825821144546e-08
Optimization successful: False
